In [52]:
import pandas as pd
import numpy as np

In [112]:
data = 'data/bureau.csv'
data_comp = 'data/proc/ft_bureau_balance.csv'

In [115]:
df_main = pd.read_csv(data, dtype={"SK_ID_CURR":str, "SK_ID_BUREAU":str})
df_comp = pd.read_csv(data_comp, dtype={"SK_ID_BUREAU":str})

In [124]:
print(df_main.shape)
print(df_comp.shape)
print(df_main.head(5))
print(df_comp.head(5))

(1716428, 17)
(817395, 9)
  SK_ID_CURR SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0     215354      5714462        Closed      currency 1         -497   
1     215354      5714463        Active      currency 1         -208   
2     215354      5714464        Active      currency 1         -203   
3     215354      5714465        Active      currency 1         -203   
4     215354      5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                   

In [117]:
# add bureau balance
df = pd.merge(df_main, df_comp, how='left', on='SK_ID_BUREAU')

In [119]:
cols_key_all = ["SK_ID_CURR", "SK_ID_BUREAU"]
cols_key_main = ["SK_ID_CURR"]
cols_ratio = []
# categoricals
cols_cate = ['CREDIT_CURRENCY', 'CREDIT_TYPE', "CREDIT_ACTIVE"]
# anything else is numerical (regular)
# numerics
cols_numeric = [k for k in df.columns if k not in cols_key_all and k not in cols_cate]

In [120]:
df_proc = df.copy().fillna(dict.fromkeys(cols_numeric, 0))

# one-hot encodings
cols_one_hot = []
for col in cols_cate:
  _df_col = pd.get_dummies(df[col], prefix='ft')
  # print(_df_col.head(5))
  for feat_col in _df_col.columns:
    df_proc[feat_col] = _df_col[feat_col]
    cols_one_hot.append(feat_col)

# do aggregation
df_agg = df_proc.groupby(main_key_cols)

numeric_methods = {"mean", "median", "max", "min"}
feature_series = {} 
columns_to_agg = cols_numeric + cols_one_hot + cols_ratio
for col in columns_to_agg:
  for method in numeric_methods:
    feature_series[f"{col}_{method}"] = df_agg[col].agg(method)

df_final = pd.DataFrame(feature_series)
for col in df_final.columns:
  _max, _min = df_final[col].max(), df_final[col].min()
  df_final[col] = df_final[col].apply(lambda x: (x - _min)/(_max - _min))

In [121]:
df_final

,DAYS_CREDIT_median,DAYS_CREDIT_min,DAYS_CREDIT_mean,DAYS_CREDIT_max,CREDIT_DAY_OVERDUE_median,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_mean,CREDIT_DAY_OVERDUE_max,DAYS_CREDIT_ENDDATE_median,DAYS_CREDIT_ENDDATE_min,...,ft_Bad debt_mean,ft_Bad debt_max,ft_Closed_median,ft_Closed_min,ft_Closed_mean,ft_Closed_max,ft_Sold_median,ft_Sold_min,ft_Sold_mean,ft_Sold_max
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0.706708,0.462012,0.748460,0.983231,0.0,0.0,0.0,0.0,0.570607,0.555994,...,0.0,0.0,1.0,0.0,0.571429,1.0,0.0,0.0,0.0,0.0
100002,0.643224,0.508214,0.700890,0.964750,0.0,0.0,0.0,0.0,0.573057,0.559502,...,0.0,0.0,1.0,0.0,0.750000,1.0,0.0,0.0,0.0,0.0
100003,0.587440,0.114990,0.520619,0.792608,0.0,0.0,0.0,0.0,0.566488,0.540910,...,0.0,0.0,1.0,0.0,0.750000,1.0,0.0,0.0,0.0,0.0
100004,0.703285,0.546201,0.703285,0.860370,0.0,0.0,0.0,0.0,0.566372,0.566013,...,0.0,0.0,1.0,1.0,1.000000,1.0,0.0,0.0,0.0,0.0
100005,0.953114,0.872348,0.934748,0.978782,0.0,0.0,0.0,0.0,0.574727,0.572388,...,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456249,0.425051,0.071526,0.429474,0.834702,0.0,0.0,0.0,0.0,0.556895,0.540023,...,0.0,0.0,1.0,0.0,0.846154,1.0,0.0,0.0,0.0,0.0
456250,0.718001,0.657084,0.704997,0.739904,0.0,0.0,0.0,0.0,0.597649,0.570422,...,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,0.0,0.0,0.0
456253,0.685489,0.685489,0.703114,0.755989,0.0,0.0,0.0,0.0,0.574412,0.571555,...,0.0,0.0,0.5,0.0,0.500000,1.0,0.0,0.0,0.0,0.0


In [122]:
df_final.to_csv("../data/proc/ft_bureau.csv")